In [2]:
import numpy as np
import matplotlib.pylab as pl
import pandas as pd
from IPython.display import display
from IPython.display import HTML

In [3]:
# [a_11, a_12, a_13]   [x]       [b_11]
# [a_21, a_22, a_33]   [y]   =   [b_21]
# [a_31, a_32, a_33] . [z]       [b_31]
#
def gaussian_elimination_no_pivoting(A: np.array, B: np.array):
    A = A.copy()
    B = B.copy()
    assert len(A.shape) == 2
    assert len(B.shape) == 1

    for i in range(A.shape[1]):
        # always start from the diag element, so as a result it'll yeild low-echelon matrix
        for j in range(i, A.shape[0]):
            el = A[j][i]
            # 1's elemination part
            if i == j:
                if el != 1:
                    A, B = row_divide(A, B, j, i)
            else:
                if el != 0:
                    A, B = rows_eliminate(A, B, j, i)
            display(pd.DataFrame(A))
    return (A, B)


# would return the new matrix with row divided by the element A_iijj
def row_divide(A: np.array, B: np.array, ii, jj):
    n = A[ii][jj]
    for j in range(len(A[ii])):
        A[ii][j] /= n
    B[ii] /= n
    return (A, B)


def rows_eliminate(A: np.array, B: np.array, ii, jj):
    A_copy = A.copy()
    B_copy = B.copy()
    support_row_index = ii + 1 if ii < A.shape[0] - 1 and A_copy[ii + 1][jj] != 0 else jj
    if A_copy[support_row_index][jj] == 0:
        raise ValueError('support value must be non-zero')
    support_row_multiplier = A_copy[ii][jj] / A_copy[support_row_index][jj]

    for j in range(len(A[support_row_index])):
        A_copy[support_row_index][j] *= support_row_multiplier
    B_copy[support_row_index] *= support_row_multiplier
    for j in range(len(A[support_row_index])):
        A[ii][j] -= A_copy[support_row_index][j]
    B[ii] -= B_copy[support_row_index]
    return (A, B)

def swap(A, b, i):
    A_copy = A.copy()
    b_copy = b.copy()
    for j in range(A_copy.shape[0]):
        if j == i:
            continue
        if A_copy[j][i] != 0 and A_copy[i][j] != 0:
            swap_buff = A_copy[j].copy()
            swap_buff_b = b_copy[j]
            A_copy[j] = A_copy[i]
            A_copy[i] = swap_buff
            b_copy[j] = b_copy[i]
            b_copy[i] = swap_buff_b
            return (A_copy, b_copy)  
    raise ValueError('row for pivoting is not found')


def gaussian_elimination_pivoting(A: np.array, b: np.array):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    for i in range(A.shape[0]):
        if A[i][i] == 0:
            A, b = swap(A, b, i)
    print('----------PIVOTING--------')
    display(pd.DataFrame(A))
    return gaussian_elimination_no_pivoting(A, b)


In [6]:
BIRTHDAY_MONTH = 1
BIRTHDAY_DAY = 2
# random_matrix = (np.random.rand(BIRTHDAY_MONTH+BIRTHDAY_DAY,
#                                 BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10 + 1).astype(int).astype(np.float32)
random_matrix = np.array([
    [6, 1, -1, 2],
    [4, 5, -3, 6],
    [-2, 3, 4, 6],
    [4, 2, -4, 2]]).astype(np.float32)
# right_side_vector_T = (np.random.rand(
#     BIRTHDAY_MONTH+BIRTHDAY_DAY) * 10).astype(int).T
right_side_vector_T = (np.array([5,9,4,2])).T
A, B = gaussian_elimination_pivoting(random_matrix, right_side_vector_T)

random_matrix_pd = pd.DataFrame(random_matrix)
right_side_vector_T_pd = pd.DataFrame(right_side_vector_T)
A_pd = pd.DataFrame(A)
B_pd = pd.DataFrame(np.array([B]).T)
print('---------------RESULT-------------')
display(random_matrix_pd)
display(right_side_vector_T_pd)
display(A_pd)
B_pd

----------PIVOTING--------


0    1    2    3
0  6.0  1.0 -1.0  2.0
1  4.0  5.0 -3.0  6.0
2 -2.0  3.0  4.0  6.0
3  4.0  2.0 -4.0  2.0

0         1         2         3
0  1.0  0.166667 -0.166667  0.333333
1  4.0  5.000000 -3.000000  6.000000
2 -2.0  3.000000  4.000000  6.000000
3  4.0  2.000000 -4.000000  2.000000

0          1         2          3
0  1.0   0.166667 -0.166667   0.333333
1  0.0  11.000000  5.000000  18.000000
2 -2.0   3.000000  4.000000   6.000000
3  4.0   2.000000 -4.000000   2.000000

0          1         2          3
0  1.0   0.166667 -0.166667   0.333333
1  0.0  11.000000  5.000000  18.000000
2  0.0   4.000000  2.000000   7.000000
3  4.0   2.000000 -4.000000   2.000000

...

---------------RESULT-------------


0    1    2    3
0  6.0  1.0 -1.0  2.0
1  4.0  5.0 -3.0  6.0
2 -2.0  3.0  4.0  6.0
3  4.0  2.0 -4.0  2.0

0
0  5
1  9
2  4
3  2

0         1         2         3
0  1.0  0.166667 -0.166667  0.333333
1  0.0  1.000000  0.454545  1.636364
2  0.0  0.000000  1.000000  0.416667
3  0.0  0.000000  0.000000  1.000000

0
0  0
1  1
2  0
3  7